In [2]:
import torch
from torch import nn
from mmcv.cnn.bricks.conv_module import ConvModule
from torchvision import models

In [40]:
class Myvgg(nn.Module):
    
    def __init__(self,num_classes=21):
        super().__init__()
        self.vgg=models.vgg16_bn(pretrained=False)
        self.vgg.features[34]=nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2),dilation=2)
        self.vgg.features[40]=nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2),dilation=2)
        self.pre =nn.Sequential(self.vgg.features[:33])
        self.mid = nn.Sequential(self.vgg.features[34:43])

        self.classifier = nn.Sequential(nn.Conv2d(512,512,1),
                                       nn.ReLU(True),
                                       nn.Conv2d(512,512,1),
                                       nn.ReLU(True),
                                       nn.Conv2d(512,num_classes,1))
    def forward(self,x):
        out =self.pre(x)
        out =self.mid(out)

  
        out = self.classifier(out)
        return out

In [41]:
my = Myvgg()

In [43]:
my.eval()
x=torch.randn(1,3,224,224)
torch.onnx.export(my,x,'test.onnx')


torch.Size([1, 512, 28, 28])
